# Comment expliquer les déserts médicaux?

## 1. Résumé

## 2. Données

### 2.1 Conversion par OCR des temps d'attentes de spécialités médicales depuis le PDF d'un article du journal _Marianne_

### 2.2 Téléchargement des données socio-démographiques depuis insee.fr

### 2.3 Extraction des données météorologiques de Wikipédia

### 2.4 Récupération de données d'attractivité des villes sur villesetvillagesouilfaitbonvivre.com

### 2.5 Difficultés liées à la consolidation des données

Notre présentation de la collecte _a posteriori_ fait faussement croire que la jointure des différentes sources de données a été chose facile. En réalité, nous avons patiemment construit un index des communes qu nous souhaitions anlayser. La constuction de cet index, mi-automatisée, mi-artisanale, nous a demandé du temps pour plusieurs raisons:

1. Il est difficile de trouver une liste des préfectures et sous-préfectures dans un format exploitable par programmation. Voici un exemple depuis Wikipedia :
    ```{python}
    import pandas as pd
    import urllib.request.urlopen as urlopen
    from bs4 import BeautifulSoup
    
    with urllib.request.urlopen("https://fr.wikipedia.org/wiki/Liste_des_pr%C3%A9fectures_de_France") as content:
        page_prefectures = BeautifulSoup(content)
    
        prefectures_sous_prefectures = page_prefectures.find('table')
        prefectures_sous_prefectures = pd.read_html(str(prefectures_sous_prefectures))
    
        # returns a list of tables of length 1
        # we lose direct URLs, departement, and population here
        prefectures_sous_prefectures = prefectures_sous_prefectures[0]    

        prefectures = prefectures_sous_prefectures['Préfecture'].to_list()
        sous_prefectures = prefectures_sous_prefectures['Sous-préfectures'].to_list()
        sous_prefectures = [[] if pd.isna(s) else s.split(", ") for s in sous_prefectures]

        # many Wikipedia pages are ambiguous :/
        prefectures_sous_prefectures = [
                "Paris"                    if ville=="Paris (C)" \
            else "Chaumont (Haute-Marne)"   if ville=="Chaumont" \
            else "Laval (Mayenne)"          if ville=="Laval" \
            else "Mende (Lozère)"           if ville=="Mende" \
            else "Moulins (Allier)"         if ville=="Moulins" \
            else "Saint-Denis (La Réunion)" if ville=="Saint-Denis" \
            else "Valence (Drôme)"          if ville=="Valence" \
            else ville for ville in prefectures
        ] + [ \
                "Pontoise" if ville=="Pontoise (Chef-lieu)" \
            else "Argenteuil (Val-d'Oise)" if ville=="Argenteuil" \
            else "Aubusson (Creuse)" if ville=="Aubusson" \
            else "Bergerac (Dordogne)" if ville=="Bergerac" \
            else "Bernay (Eure)"             if ville=="Bernay" \
            else "Bonneville (Haute-Savoie)" if ville=="Bonneville" \
            else "Clamecy (Nièvre)" if ville=="Clamecy" \
            else "Clermont (Oise)" if ville=="Clermont" \
            else "Cognac (Charente)" if ville=="Cognac" \
            else "Condom (Gers)" if ville=="Condom" \
            else "Die (Drôme)" if ville=="Die" \
            else "Dieppe (Seine-Maritime)" if ville=="Dieppe" \
            else "Dole (Jura)" if ville=="Dole" \
            else "Gourdon (Lot)" if ville=="Gourdon" \
            else "La Trinité (Martinique)" if ville=="La Trinité" \
            else "Langon (Gironde)" if ville=="Langon" \
            else "Le Blanc (Indre)" if ville=="Le Blanc" \
            else "Le Vigan (Gard)" if ville=="Le Vigan" \
            else "Lens (Pas-de-Calais)" if ville=="Lens" \
            else "Mauriac (Cantal)" if ville=="Mauriac" \
            else "Mayenne (commune)" if ville=="Mayenne" \
            else "Montdidier (Somme)" if ville=="Montdidier" \
            else "Montreuil (Pas-de-Calais)" if ville=="Montreuil" \
            else "Muret (Haute-Garonne)" if ville=="Muret" \
            else "Neufchâteau (Vosges)" if ville=="Neufchâteau" \
            else "Prades (Pyrénées-Orientales)" if ville=="Prades" \
            else "Péronne (Somme)" if ville=="Péronne" \
            else "Rochechouart (Haute-Vienne)" if ville=="Rochechouart" \
            else "Rochefort (Charente-Maritime)" if ville=="Rochefort" \
            else "Saint-Benoît (La Réunion)" if ville=="Saint-Benoît" \
            else "Saint-Claude (Jura)" if ville=="Saint-Claude" \
            else "Saint-Denis (Seine-Saint-Denis)" if ville=="Saint-Denis" \
            else "Saint-Flour (Cantal)" if ville=="Saint-Flour" \
            else "Saint-Girons (Ariège)" if ville=="Saint-Girons" \
            else "Saint-Omer (Pas-de-Calais)" if ville=="Saint-Omer" \
            else "Saint-Paul (La Réunion)" if ville=="Saint-Paul" \
            else "Senlis (Oise)" if ville=="Senlis" \
            else "Sens (Yonne)" if ville=="Sens" \
            else "Torcy (Seine-et-Marne)" if ville=="Torcy" \
            else "Ussel (Corrèze)" if ville=="Ussel" \
            else "Vienne (Isère)" if ville=="Vienne" \
            else ville for villes in sous_prefectures for ville in villes if ville != "Saint-Pierre"\
        ] + [
            "Saint-Pierre (Martinique)", "Saint-Pierre (La Réunion)"
        ]

    prefectures_sous_prefectures.sort()
    prefectures_sous_prefectures
    ```
2. L'hebdomadaire _Marianne_ ne respecte pas ses propres contraintes et en particulier (a) a enquêté des ville squi ne sont ni des préfecrtues ni des sous-préfectures, (b) a négligé entièrement les préfectures et sous-préfectures d'Outre-Mer.
3. Le site villesetvillagesouilfaitbonvivre.com nécessite le code postal pour récupérer les données, ce quis est pourtant inutile puisque le code INSEE est un identifiant des municipalités.
4. Les URL de Wikipédia ne sont pas non plus facilement accessible via le code INSEE, et il aura fallu résoudre toutes les ambiguités de nommage pour de nombreuses villes.


## 3. Analyse descriptive

### 3.1 Pas de structure en basse dimension des temps d'attente des différentes spécialités

En l'absence de structure particulière, et étant limité dans nos efforts d'analyses, nous choisissons par simplification à la section 4 de nous restreindre à l'analyse du **temps d'attente chez le généraliste**.

### 3.2 Pas de motif géographique clair dans les temps d'attente < À CONFIRMER >

### 3.3 < LIAISON ENTRE LES TEMPS D'ATTENTES ET LES AUTRES VARIABLES >

## 4. Modélisation

Dans cette section, nous nous restreignons par soucis de simplification à l'analyse du **temps d'attente chez le généraliste**.

### 4.1 < RÉGRESSION >

### 4.2 < CARTE DES RÉSIDUS >

## 5. Démarche scientifique et reproductibilité

## 6. Format du code

Le rapport est accessible [sur Deepnote](https://deepnote.com/project/desert-medical-L4EiMo7RTomPSFl-EAWGNw/%2Freport.ipynb), où il est exécutable. La totalité du code et des données sont par ailleurs disponibles [sur Github](https://github.com/katossky/desert-medical). L'historique des enregistrements (_commits_) est disponible [ici](https://github.com/katossky/desert-medical/commits/main). Il n'y a pas eu lieu, vu la simplicité relative du projet, d'avoir recours à des branches de développement.

La plupart du code a été reporté en annexe, mais le rapport contient un exemple filé exécutable pour la ville de Poitiers. Le code est abondamment commenté et nous avons porté un soin particulier au choix des noms de variables afin d'en faciliter la lecture.

## 7. Pistes d'amélioration

- Étendre la modélisation à l'ensemble des temps d'attentes, séparément dans un premier temps, puis conjointement.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f812232-8ed1-4e89-8f48-597e10058637' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>